In [1]:
import wikipedia
import transformers 
import pandas as pd
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [2]:
df = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\sample_ds\sample_set.v1.json', lines=True) #Cambiar para directorio local
df.head(5)

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels
0,1,EN,When was the restoration of the Sándor Palace ...,"The restoration of Sándor Palace, also known ...",TheBloke/Mistral-7B-Instruct-v0.2-GGUF,"[{'start': 33, 'prob': 0.33333333330000003, 'e...","[[53, 64], [151, 164], [165, 210], [211, 225],..."
1,3,EN,Did the Scotland team debut in the UEFA Euro 1...,"Yes, Scotland made their debut in the UEFA Eu...",TheBloke/Mistral-7B-Instruct-v0.2-GGUF,"[{'start': 1, 'prob': 0.6666666667000001, 'end...","[[1, 4], [49, 53], [87, 103], [232, 246], [262..."
2,4,FR,Contre quelles équipes l'Azerbaïdjan a-t-il ét...,L'équipe d'Azerbaïdjan n'a pas participé à l'...,TheBloke/Mistral-7B-Instruct-v0.2-GGUF,"[{'start': 1, 'prob': 0.33333333330000003, 'en...","[[24, 55], [60, 162], [213, 320]]"
3,8,FR,8437 Bernicla est-il un objet transneptunien ?,8437 Bernicla est-il un objet transneptunien?\...,meta-llama/Meta-Llama-3-8B-Instruct,"[{'start': 46, 'prob': 0.33333333330000003, 'e...","[[71, 86], [90, 94], [184, 193], [269, 437], [..."
4,9,EN,Over which state did Xiong Ai rule?,Xiong Ai was a Chinese warlord who lived duri...,TheBloke/Mistral-7B-Instruct-v0.2-GGUF,"[{'start': 10, 'prob': 0.33333333330000003, 'e...","[[24, 31], [42, 48], [75, 87], [92, 123], [140..."


In [3]:
# Extracción de preguntas.
a = []
for i in range(len(df)):
    if df.loc[i]["lang"] == "EN":
        a.append(df.loc[i]["model_input"])
print(a)

['When was the restoration of the Sándor Palace completed?', 'Did the Scotland team debut in the UEFA Euro 1996 qualifying phase?', 'Over which state did Xiong Ai rule?']


In [10]:
# FUNCIONES

# a = list; Lista de preguntas extraída del dataset.
# Idealmente esa lista contiene todas las preguntas necesarias.
# Regresa la lista de las palabras filtradas por PoST
def noun_list(a):
    #question_words = ["How", "how", "Where", "where", "When", "when", "What", "what", "Why", "why", "Who", "who", "Which", "which"]
    post_spacy = spacy.load("en_core_web_sm")
    noun_list = []

    for _ in a:
        doc = post_spacy(_)
        sub_noun = []
        for token in doc:
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "NUM":
                sub_noun.append(token.text)
            #if token.text in question_words:  # Estas partes se descomentan en caso de que se busque agregar las palabras de pregunta.
            #    sub_noun.append(token.text)
        noun_list.append(sub_noun)
    return noun_list

# n_list = list;  Lista obtenida con la función noun_list(). Debe de contener las preguntas preprocesadas.
def keyword_por_preg(n_list):
    keyword_list = []
    for i in n_list:
        keyword = ''
        for j in i:
            keyword = keyword + j + ' '
        keyword_list.append(keyword)
    return keyword_list


# Esta función regresa el título de la página de wikipedia más relevante.
# text = str ; El texto proveniente de keyword_list.
def get_wikipage(text):
    page_title = wikipedia.search(text, results = 1)
    return wikipedia.page(page_title)

In [11]:
lista_prueba = noun_list(a)
key_list = keyword_por_preg(lista_prueba)
print(lista_prueba)
print(key_list)

[['restoration', 'Sándor', 'Palace'], ['Scotland', 'team', 'debut', 'UEFA', 'Euro', '1996', 'qualifying', 'phase'], ['state', 'Xiong', 'Ai', 'rule']]
['restoration Sándor Palace ', 'Scotland team debut UEFA Euro 1996 qualifying phase ', 'state Xiong Ai rule ']


In [12]:
for i in key_list:
    print(get_wikipage(i).content)
    print("-----------------------------------------------------------------")

Sándor Palace (Hungarian: Sándor-palota, pronounced [ˈʃaːndor ˈpɒlotɒ]) is a palace in Budapest, Hungary. Located beside the Buda Castle complex in the ancient Castle District, it has served as the official residence and workspace of the president of Hungary since 2003.
Sándor Palace is the 37th largest palace in present-day Hungary.


== History ==


=== 19th century ===
The original Neoclassical style palace was begun about 1803, and completed about 1806.  Count Vincent Sándor commissioned it, and it was named after him.  Count Vincent Sándor was a philosopher and aristocrat in the Austrian Empire. His son Count Móric Sándor de Szlavnicza (1805–1878) was better known in Budapest and Vienna, from fame for acrobatic jousts.
The palace then next belonged to Archduke Albrecht, the Imperial Governor of Hungary, until the failed Hungarian Revolution of 1848. After that the palace and its adjacent buildings facing the square were rented as government offices.

Prime ministers
The most prest

Una idea inicial es obtener un resumen proveniente de Wikipedia para poder usar un modelo de verificación de halucinaciones como:
https://huggingface.co/vectara/hallucination_evaluation_model
O hacer un finetuning de:
https://huggingface.co/FacebookAI/xlm-roberta-large


Wikipedia API Docs: https://wikipedia.readthedocs.io/en/latest/code.html

**A partir de aquí no se usa el código**

----------------------------------------------------------------

In [4]:
# Búsqueda de la página de wikipedia correspondiente a la pregunta entera
# Es mejor hacer un filtro por PoST y luego realizar esto
for _ in a:
    search = wikipedia.search(_, results=5)
    aux = search[0]
    print(aux, search)
    print(wikipedia.summary(aux))
    print("----")

Sándor-Metternich mansion ['Sándor-Metternich mansion', 'Edirne Palace', 'Jal Mahal', 'Palace of Westminster', 'Big Ben']
The Sándor-Metternich mansion (Hungarian: Sándor–Metternich-kastély) is a classicist manor in Bajna in Esztergom County, Hungary. It lies approximately 50 kilometres northwest of Budapest. It was the main country residence of the Metternich-Sándor family. It has been restored from ruins with the help of the European Union and is open for visitors.


----
UEFA Euro 1996 qualifying ['UEFA Euro 1996 qualifying', 'UEFA Euro 2024 qualifying', 'UEFA Euro 1992 qualifying', "UEFA Women's Euro 2025 qualifying", 'UEFA Euro 2024']
Qualifying for UEFA Euro 1996 took place throughout 1994 and 1995. Forty-seven teams were divided into eight groups, with each team playing the others in their group both home and away. The winners of each group and the six best runners-up qualified automatically, while the two worst runners-up were involved in a play-off at a neutral venue. England 

In [6]:
# Modelo NER 
# Se busca encontrar conceptos clave para poder hacer una búsqueda paralela

tokenizer_ner = AutoTokenizer.from_pretrained('dslim/bert-base-NER')
model_ner = AutoModelForTokenClassification.from_pretrained('dslim/bert-base-NER')

nlp_ner = pipeline('ner', model = model_ner, tokenizer = tokenizer_ner)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [20]:
# NER Usando modelo preentrenado
for _ in a:
    ner_res = nlp_ner(_)
    print(_)
    print(f"Length NER: {len(ner_res)}")
    for i in ner_res:
        print(i)
    print(ner_res[0]["word"])
    print("---")

When was the restoration of the Sándor Palace completed?
Length NER: 4
{'entity': 'B-LOC', 'score': 0.9837886, 'index': 7, 'word': 'S', 'start': 32, 'end': 33}
{'entity': 'I-LOC', 'score': 0.75637233, 'index': 8, 'word': '##án', 'start': 33, 'end': 35}
{'entity': 'I-LOC', 'score': 0.8117068, 'index': 9, 'word': '##dor', 'start': 35, 'end': 38}
{'entity': 'I-LOC', 'score': 0.8922695, 'index': 10, 'word': 'Palace', 'start': 39, 'end': 45}
S
---
Did the Scotland team debut in the UEFA Euro 1996 qualifying phase?
Length NER: 4
{'entity': 'B-LOC', 'score': 0.9998394, 'index': 3, 'word': 'Scotland', 'start': 8, 'end': 16}
{'entity': 'B-MISC', 'score': 0.9992681, 'index': 8, 'word': 'UEFA', 'start': 35, 'end': 39}
{'entity': 'I-MISC', 'score': 0.99482286, 'index': 9, 'word': 'Euro', 'start': 40, 'end': 44}
{'entity': 'I-MISC', 'score': 0.9990723, 'index': 10, 'word': '1996', 'start': 45, 'end': 49}
Scotland
---
Over which state did Xiong Ai rule?
Length NER: 3
{'entity': 'B-PER', 'score': 0.9